In [ ]:
import pandas as pd
import torch
import numpy as np
from vllm import LLM, SamplingParams
from tqdm.auto import tqdm
import random
import os

import os
from openai import OpenAI

token = 'Your API Key'

os.environ["OPENAI_API_KEY"] = token

2024-11-04 12:42:14.732224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730691734.749731  423556 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730691734.755015  423556 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 12:42:14.776384: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rihankoo/.local/lib/python3.11/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No modu

In [ ]:
# seed = 777
# seed = 20241027
seed = 20240617
# seed = 355643

def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')

set_seed(seed)

> SEEDING DONE


In [ ]:
df = pd.read_csv('../../../../data/raw/misconception_mapping.csv')
df

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...
...,...,...
2582,2582,"When multiplying numbers with the same base, m..."
2583,2583,Does not know what a cube number is
2584,2584,Believes that any percentage of a larger numbe...
2585,2585,Believes a cubic expression should have three ...


In [ ]:
import pandas as pd
df_train = pd.read_csv('../../../../data/1.chain_of_thought/train.csv')
df_train

,QuestionId_Answer,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,Answer,AnswerText,Correct,CorrectAnswer,CorrectAnswerText,FirstSubjectId,FirstSubjectName,SecondSubjectId,SecondSubjectName,ThirdSubjectId,ThirdSubjectName,MisconceptionId,p000-qwen25-32b-instruct-cot_misunderstanding
0,0_D,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,D,Does not need brackets,0,A,\( 3 \times(2+4)-5 \),32,Number,144,Basic Arithmetic,33,BIDMAS,1672.0,The students' misunderstanding lies in their l...
1,1_A,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",A,\( m+1 \),0,D,Does not simplify,49,Algebra,255,Algebraic Fractions,1077,Simplifying Algebraic Fractions,2142.0,The misunderstanding likely stems from a conce...
2,1_B,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",B,\( m+2 \),0,D,Does not simplify,49,Algebra,255,Algebraic Fractions,1077,Simplifying Algebraic Fractions,143.0,The misunderstanding likely stems from a conce...
3,1_C,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",C,\( m-1 \),0,D,Does not simplify,49,Algebra,255,Algebraic Fractions,1077,Simplifying Algebraic Fractions,2142.0,### Explanation of the Mistake\n\n**Step-by-St...
4,2_A,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,A,Only\nTom,0,B,Only\nKatie,101,Data and Statistics,338,Data Processing,339,Range and Interquartile Range from a List of Data,1287.0,The misunderstanding here revolves around how ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4365,1867_C,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,Tom and Katie are discussing congruence and si...,C,Both Tom and Katie,0,B,Only Katie,71,Geometry and Measure,272,Similarity and Congruency,274,Congruency in Other Shapes,2312.0,The misunderstanding here revolves around the ...
4366,1867_D,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,Tom and Katie are discussing congruence and si...,D,Neither is correct,0,B,Only Katie,71,Geometry and Measure,272,Similarity and Congruency,274,Congruency in Other Shapes,2312.0,The misunderstanding here revolves around the ...
4367,1868_A,1868,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,Jo and Paul are arguing about how to fully des...,A,Only\nJo,0,B,Only Paul,71,Geometry and Measure,278,Transformations,93,Rotation,801.0,The misunderstanding here revolves around the ...
4368,1868_C,1868,2680,Describe a 90° or 270° rotation giving the ang...,93,Rotation,Jo and Paul are arguing about how to fully des...,C,Both Jo and Paul,0,B,Only Paul,71,Geometry and Measure,278,Transformations,93,Rotation,801.0,The misunderstanding here revolves around the ...


In [8]:
system_prompt_template = 'You are an excellent math teacher about to teach students of year group 1 to 14. You will be provided a misconception that your students may have. Please write 5 diagnostic questions that can help you identify the misconception. Only write the question. No need to provide explanation for correct answer. Each question should be in format of "Topic: {subject}. Question: {question}."'
example_template = 'Misconception: {misconception}\nTopic: {subject}. Question: {question}.'
user_prompt_template = "Here are some examples:\n{examples}.\nHere are all of the topics of your lesson: {subjects}\n\nPlease start writing the question. Misconception: {misconception}\n"

In [ ]:
messages_list = []
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    misconception = row['MisconceptionName']

    df_train_sample = df_train.sample(few_shot).reset_index(drop=True)
    examples = []
    for j in range(len(df_train_sample)):
        row_sample = df_train_sample.iloc[j]
        misconception_sample = row_sample['Misconception']
        question_sample = row_sample['QuestionText']
        correct_answer_sample = row_sample['CorrectAnswerText']
        wrong_answer_sample = row_sample['AnswerText']
        first_subject = row_sample['FirstSubjectName']
        second_subject = row_sample['SecondSubjectName']
        third_subject = row_sample['ThirdSubjectName']
        construct = row_sample['ConstructName']
        misunderstanding = row_sample['p000-qwen25-32b-instruct_misunderstanding']

        example = f'example{j+1}: \n' + example_template.format(misconception=misconception_sample, subject=third_subject, question=question_sample)
        examples.append(example)

    example_str = '\n\n'.join(examples)

    user_prompt = user_prompt_template.format(misconception=misconception, examples=example_str, subjects=all_subjects_str)
    system_prompt = system_prompt_template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    messages_list.append(messages)

  0%|          | 0/2587 [00:00<?, ?it/s]

In [ ]:
from typing import List
import json

output_file = "train_question_generation1.jsonl"

with open(output_file, 'a') as f:
    _id=0
    for messages in messages_list[2400:]:
        data = {
            "custom_id": f"request-{_id}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": messages,
                "max_tokens": 10000,
            }
        }
        _id+=1
        f.write(json.dumps(data) + '\n')

In [ ]:


client = OpenAI()

batch_input_file = client.files.create(
  file=open(output_file, "rb"),
  purpose="batch"
)

In [ ]:
batch_input_file_id = batch_input_file.id

batch_job = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "train_question_generation1"
    }
)

## wait for the batch job to complete and fetch the result

In [ ]:
client.batches.retrieve(batch_job.id)

In [ ]:
import json
output_file_id = "your-output-file-id"
file_response = client.files.content(output_file_id)
lines = file_response.text.strip().split('\n')
data = [json.loads(line) for line in lines]
len(data)

In [ ]:
df['synthetic_question'] = data

In [5]:
def extract_question(text):
    topics = []
    questions = []
    splits = text.split('Topic:')
    for split in splits:
        try:
            topic, question = split.split('Question:')
            topic = topic.strip()
            if topic[-1] == '.':
                topic = topic[:-1]
            question = question.strip()
            if question[-1] == '.':
                question = question[:-1]
            topics.append(topic)
            questions.append(question)
        except:
            continue
    return topics, questions

In [ ]:
rows = []
for i in range(len(df)):
    row = df.iloc[i]
    mis_id = row['MisconceptionId']
    mis_name = row['MisconceptionName']
    synthetic_question = row['synthetic_question']
    topics, questions = extract_question(synthetic_question)
    for topic, question in zip(topics, questions):
        rows.append([mis_id, mis_name, topic, question])
df_generated = pd.DataFrame(rows, columns=['MisconceptionId', 'MisconceptionName', 'SubjectName', 'QuestionText'])

In [ ]:
df_generated.to_csv('../../../../data/2.synthetic_data_generation/generation1/generation1_openai.csv', index=False)